Pliki \*\_translated powstały poprzez przetłumaczenie postów z wykorzystaniem LibreTranslate (translate.ipynb)

In [20]:

import csv

import pandas as pd

with open('Lab-3-Zadanie-3-Dane Mandrill Przetłumaczone.csv', 'r') as f:
    reader = csv.reader(f)
    mandrill_data = list(x[0] for x in reader if x != [])
    mandrill_data.pop(0)

with open('Lab-3-Zadanie-3-Dane Inne Przetłumaczone.csv', 'r') as f:
    reader = csv.reader(f)
    other_data = list(x[0] for x in reader if x != [])
    other_data.pop(0)


In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('words')
porter = PorterStemmer()

english_words = nltk.corpus.words.words()
def tokenize(text: str) -> list:
    # Usunięcie linków, tagów @ na początku
    text = " ".join([x for i, x in enumerate(text.split(' ')) if not x.startswith('http') and not (i == 0 and x.startswith('@'))])

    # Tokenyzacja
    tokenizer = RegexpTokenizer(r'[A-Za-z_\']+') # Regex usuwający interpunkcje
    tokens = tokenizer.tokenize(text.lower())

    # Usunięcie wyrazów przystankowych
    stop_words = set(stopwords.words('english')) # angielskie wyrazy przystankowe
    filtered_tokens = set([porter.stem(word) for word in tokens if word not in stop_words and len(word) >= 2])
    return filtered_tokens

def tokenize_lines(lines_of_text: list[str]) -> tuple[list, set]:
    text_tokens = []
    unique_tokens = set()
    for line in lines_of_text:
        filtered_tokens = tokenize(line)
        text_tokens.append(filtered_tokens)
        unique_tokens.update(filtered_tokens)
    return text_tokens, unique_tokens



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Szymon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Szymon\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [22]:
mandrill_tokens, unique_tokens = tokenize_lines(mandrill_data)
other_tokens, temp = tokenize_lines(other_data)

unique_tokens.update(temp)

data = pd.DataFrame({'clean_text': [], 'text': [], 'label': []})
mandrill_df = pd.DataFrame({'text': mandrill_data[i],'clean_text': " ".join(tokens), 'label': 'Mandrill'} for i, tokens in enumerate(mandrill_tokens))
other_df = pd.DataFrame({'text': other_data[i],'clean_text': " ".join(tokens), 'label': 'Inne'} for i, tokens in enumerate(other_tokens))

data = pd.concat([mandrill_df, other_df])
data


,text,clean_text,label
0,[blog] Using Nullmailer and Mandrill for your ...,server blog use mandril linux ubuntu outboud n...,Mandrill
1,[blog] Using Postfix and free Mandrill email s...,free server blog use mandril servic linux post...,Mandrill
2,@aalbertson There are several reasons emails g...,addit request mind reason go sever detail subm...,Mandrill
3,@adrienneleigh I just switched it over to Mand...,let' switch sent mandril see improv speed email,Mandrill
4,@ankeshk +1 to @mailchimp We use MailChimp for...,market mailchimp hiway app use mandril abhijee...,Mandrill
...,...,...,...
144,Why Are Monkey Butts So Colorful?: Mandrill Wi...,monkey commonsplu wikimedia mandril butt popul...,Inne
145,You can now experience the thrills of classic ...,thrill classic pc ehh know maze mandril game e...,Inne
146,??????? ? Mainland?75????? Oddisee??????B3?Sil...,oddise mandril silk mainland solid,Inne
147,???????????????????????????????B1???80????????...,mandril mood get,Inne


In [23]:
from sklearn.model_selection import train_test_split

X = data["clean_text"]
y = data["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, stratify=y
)


In [24]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [25]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_vec, y_train)


,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

y_pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nMacierz pomyłek:")
print(confusion_matrix(y_test, y_pred, labels=['Inne', 'Mandrill']))

precision = (precision_score(y_test, y_pred, pos_label='Inne'), precision_score(y_test, y_pred, pos_label='Mandrill'))
recall = (recall_score(y_test, y_pred, pos_label='Inne'), recall_score(y_test, y_pred, pos_label='Mandrill'))

print( "|           |   Inne   | Mandrill |")
print( "|===========+==========+==========|")
print(f"| Precision |   {precision[0]:4.2f}   |   {precision[1]:4.2f}   |")
print(f"| Recall    |   {recall[0]:4.2f}   |   {recall[1]:4.2f}   |")



Accuracy: 0.875

Macierz pomyłek:
[[48 12]
 [ 3 57]]
|           |   Inne   | Mandrill |
|===========+==========+==========|
| Precision |   0.94   |   0.83   |
| Recall    |   0.80   |   0.95   |


### Testy na syntetycznych danych

In [33]:
# Przykładowe posty wraz z oceną - wygenerowane przez LLM
test_posts = [
    ("Email delivery stopped right after we changed the mandrill settings", "Mandrill"),
    ("Not sure if mandrill supports this kind of webhook event", "Mandrill"),
    ("Our backend logs show mandrill rejecting messages again", "Mandrill"),
    ("The issue seems related to mandrill authentication rather than our code", "Mandrill"),
    ("Spent the whole night debugging mandrill responses from the API", "Mandrill"),
    ("Mandrill finally processed the batch after several retries", "Mandrill"),
    ("We thought it was our server, but mandrill was actually down", "Mandrill"),
    ("Anyone else seeing delayed notifications when using mandrill?", "Mandrill"),
    ("After switching providers, mandrill still handles receipts better", "Mandrill"),
    ("Mandrill errors are not very descriptive in the logs", "Mandrill"),
    ("The mandrill in the enclosure barely moved all afternoon", "Inne"),
    ("At first I thought mandrill was a tool, but it turned out to be an animal", "Inne"),
    ("That mandrill looked surprisingly calm compared to others", "Inne"),
    ("The guide explained how mandrill groups communicate in the wild", "Inne"),
    ("I confused mandrill with baboon during the lecture", "Inne"),
    ("A mandrill appeared briefly in the nature documentary", "Inne"),
    ("The zoo updated the mandrill habitat last year", "Inne"),
    ("Children were fascinated by the mandrill’s colorful face", "Inne"),
    ("Mandrill behavior changes significantly in captivity", "Inne"),
    ("Scientists observed mandrill migration patterns", "Inne"),
    ("Mandrill documentation is not always up to date", "Mandrill"),
    ("We removed mandrill integration after repeated failures", "Mandrill"),
    ("Mandrill messages succeed locally but fail in production", "Mandrill"),
    ("Hard to tell if mandrill throttling caused the delay", "Mandrill"),
    ("Mandrill metrics look fine but emails are missing", "Mandrill"),
    ("Someone mentioned mandrill but they meant the animal", "Inne"),
    ("Mandrill populations are declining due to habitat loss", "Inne"),
    ("I searched for mandrill and got both code and wildlife results", "Inne"),
    ("Mandrill appeared in both my biology and programming notes", "Inne"),
    ("The word mandrill caused confusion during the discussion", "Inne"),
    ("Mandrill webhooks triggered twice for the same event", "Mandrill"),
    ("We rely on mandrill for all transactional traffic", "Mandrill"),
    ("Mandrill seems stable now after the earlier outage", "Mandrill"),
    ("Logs suggest mandrill rejected the payload format", "Mandrill"),
    ("A mandrill stared back at us through the glass", "Inne"),
    ("Mandrill facial coloration plays a role in dominance", "Inne"),
    ("The mandrill suddenly charged toward the fence", "Inne"),
    ("Mandrill is often mistaken for other primates", "Inne"),
    ("Reading about mandrill ecology was surprisingly interesting", "Inne"),
    ("Is Mandrill down right now?", "Mandrill")
]

synthetic_data = pd.DataFrame(
    {'text': post[0], 'clean_text': " ".join(tokenize(post[0])), 'label': post[1]} for post in test_posts
)
synthetic_x_test_vec = vectorizer.transform(synthetic_data['clean_text'])
synthetic_y_test = synthetic_data['label']
synthetic_y_pred = model.predict(synthetic_x_test_vec)

print("Accuracy:", accuracy_score(synthetic_y_test, synthetic_y_pred))
print("\nMacierz pomyłek:")
print(confusion_matrix(synthetic_y_test, synthetic_y_pred, labels=['Inne', 'Mandrill']))

synthetic_data_precision = (precision_score(synthetic_y_test, synthetic_y_pred, pos_label='Inne'), precision_score(synthetic_y_test, synthetic_y_pred, pos_label='Mandrill'))
synthetic_data_recall = (recall_score(synthetic_y_test, synthetic_y_pred, pos_label='Inne'), recall_score(synthetic_y_test, synthetic_y_pred, pos_label='Mandrill'))

print( "|           |   Inne   | Mandrill |")
print( "|===========+==========+==========|")
print(f"| Precision |   {synthetic_data_precision[0]:4.2f}   |   {synthetic_data_precision[1]:4.2f}   |")
print(f"| Recall    |   {synthetic_data_recall[0]:4.2f}   |   {synthetic_data_recall[1]:4.2f}   |")

Accuracy: 0.775

Macierz pomyłek:
[[15  5]
 [ 4 16]]
|           |   Inne   | Mandrill |
|===========+==========+==========|
| Precision |   0.79   |   0.76   |
| Recall    |   0.75   |   0.80   |


In [35]:
for i, pred in enumerate(synthetic_y_pred):
    print(synthetic_data['text'][i])
    print(f'Prognoza: {pred} | Rzeczywista klasa: {synthetic_y_test[i]}')
    print('\n')

Email delivery stopped right after we changed the mandrill settings
Prognoza: Mandrill | Rzeczywista klasa: Mandrill


Not sure if mandrill supports this kind of webhook event
Prognoza: Mandrill | Rzeczywista klasa: Mandrill


Our backend logs show mandrill rejecting messages again
Prognoza: Inne | Rzeczywista klasa: Mandrill


The issue seems related to mandrill authentication rather than our code
Prognoza: Mandrill | Rzeczywista klasa: Mandrill


Spent the whole night debugging mandrill responses from the API
Prognoza: Mandrill | Rzeczywista klasa: Mandrill


Mandrill finally processed the batch after several retries
Prognoza: Mandrill | Rzeczywista klasa: Mandrill


We thought it was our server, but mandrill was actually down
Prognoza: Mandrill | Rzeczywista klasa: Mandrill


Anyone else seeing delayed notifications when using mandrill?
Prognoza: Mandrill | Rzeczywista klasa: Mandrill


After switching providers, mandrill still handles receipts better
Prognoza: Mandrill | Rzeczywist